In [1]:
import nibabel as nb
import numpy as np
import scipy.io
import scipy
from scipy import stats
import math

In [2]:
mat_left = scipy.io.loadmat('D:/place_ROIs_project/Movie_watching_avg_data/notmean_left_Meanfile.mat')
mat_right = scipy.io.loadmat('D:/place_ROIs_project/Movie_watching_avg_data/notmean_right_Meanfile.mat')

In [3]:
corr_left_dict = mat_left
corr_right_dict = mat_right

In [4]:
left_movie_data = mat_left['Left_data']
right_movie_data = mat_right['Right_data']

In [5]:
FPPs_right = (nb.load('D:/place_ROIs_project/Phase1_ExtractingFinalROIs/1_Finding_ROIs/sub_2/FPPs.rh.func.gii').darrays[0].data).T
FPPs_left = (nb.load('D:/place_ROIs_project/Phase1_ExtractingFinalROIs/1_Finding_ROIs/sub_2/FPPs.lh.func.gii').darrays[0].data).T

In [6]:
right_FPPs_indices = [np.nonzero(FPPs_right[i]) for i in range(5)]
left_FPPs_indices = [np.nonzero(FPPs_left[i]) for i in range(5)]

In [7]:
activity_of_FPPs = [np.concatenate((right_movie_data[right_FPPs_indices[i]], left_movie_data[left_FPPs_indices[i]])) for i in range(5)]

In [8]:
mean_FPPs_activity = np.array([np.mean(activity_of_FPPs[i], axis=0) for i in range(5)])

In [13]:
mean_FPPs_activity.shape

(5, 3655)

# Right

In [10]:
right_movie_data.shape

(32492, 3655)

In [11]:
correlation_right_vertices_FPPs = np.zeros((32492, 5))
pvalue_right_vertices_FPPs = np.zeros((32492, 5))

for i in range(right_movie_data.shape[0]):
    for j in range(5):
        corr,p_value = stats.pearsonr(right_movie_data[i], mean_FPPs_activity[j])
        correlation_right_vertices_FPPs[i,j] = corr
        pvalue_right_vertices_FPPs[i,j] = p_value

C:\Users\roza\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [26]:
correlation_right_vertices_FPPs = np.nan_to_num(correlation_right_vertices_FPPs)
pvalue_right_vertices_FPPs = np.nan_to_num(pvalue_right_vertices_FPPs)
corr_right_32 = correlation_right_vertices_FPPs.astype(np.float32)
pvalue_right_32 = pvalue_right_vertices_FPPs.astype(np.float32)

In [44]:
corr_right_32.shape

(32492, 5)

In [38]:
temp_r_path = 'D:/place_ROIs_project/Initial_ROIs_data/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
for i in range(1,6):
    temp_r.darrays[0].data = corr_right_32[:,i-1]
    temp_r.to_filename(f'D:/place_ROIs_project/Phase1_ExtractingFinalROIs/2_Correlation_vertices_with_FPPs/CorrelationFPPsWithVertices{i}.rh.func.gii')

In [13]:
sorted_correlation_right_vertices_FPPs = np.sort(correlation_right_vertices_FPPs, axis=1)[:, ::-1]
sorted_pvalue_right_vertices_FPPs = np.sort(pvalue_right_vertices_FPPs, axis=1)[:, ::-1]
sorted_prefered_FPP_each_vertices = np.argsort(correlation_right_vertices_FPPs, axis=1)[:, ::-1]

In [14]:
preferred_FPP_each_vertices = sorted_prefered_FPP_each_vertices[:,0] + 1

In [15]:
preferred_FPP_each_vertices

array([5, 5, 5, ..., 5, 5, 5], dtype=int64)

In [16]:
positive_preferred_corr_each_vertices_right = sorted_correlation_right_vertices_FPPs[:,0][sorted_correlation_right_vertices_FPPs[:,0]> 0]

In [17]:
mean_sorted_correlation_right_vertices_FPPs_right = np.mean(positive_preferred_corr_each_vertices_right) 

In [18]:
mean_sorted_correlation_right_vertices_FPPs_right

0.3510572207193558

In [70]:
threshold = 0.3
above_threshold_vertices = sorted_correlation_right_vertices_FPPs[:,0] >= threshold

In [71]:
above_threshold_vertices

array([False, False, False, ..., False, False,  True])

In [72]:
preferred_FPP_above_threshold_vertices = np.zeros_like(preferred_FPP_each_vertices)

preferred_FPP_above_threshold_vertices[above_threshold_vertices] = preferred_FPP_each_vertices[above_threshold_vertices]

In [73]:
preferred_FPP_above_threshold_vertices

array([0, 0, 0, ..., 0, 0, 5], dtype=int64)

In [74]:
temp_r_path = 'D:/place_ROIs_project/Initial_ROIs_data/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
temp_r.darrays[0].data = preferred_FPP_above_threshold_vertices
temp_r.to_filename('D:/place_ROIs_project/Phase1_ExtractingFinalROIs/2_Correlation_vertices_with_FPPs/preferred_FPP_above_threshold_vertices.R.func.gii')

In [31]:
mean_FPPs_activity

array([[ 0.11503519,  0.03718783,  0.06390971, ..., -0.09333008,
        -0.04491226, -0.01514777],
       [ 0.08077597, -0.02147286,  0.03186865, ..., -0.16496121,
        -0.13001685, -0.07163416],
       [-0.04819481, -0.19603622, -0.02925085, ..., -0.48429933,
        -0.47145943, -0.40271844],
       [-0.12274933, -0.24787016, -0.14828766, ..., -0.43621268,
        -0.43554419, -0.40134424],
       [-0.31982471, -0.23964374, -0.09059742, ..., -0.37779491,
        -0.33173222, -0.25312564]])

# Left

In [45]:
correlation_left_vertices_FPPs = np.zeros((32492, 5))
pvalue_left_vertices_FPPs = np.zeros((32492, 5))

for i in range(left_movie_data.shape[0]):
    for j in range(5):
        corr,p_value = stats.pearsonr(left_movie_data[i], mean_FPPs_activity[j])
        correlation_left_vertices_FPPs[i,j] = corr
        pvalue_left_vertices_FPPs[i,j] = p_value

In [46]:
correlation_left_vertices_FPPs = np.nan_to_num(correlation_left_vertices_FPPs)
pvalue_left_vertices_FPPs = np.nan_to_num(pvalue_left_vertices_FPPs)
corr_left_32 = correlation_left_vertices_FPPs.astype(np.float32)
pvalue_left_32 = pvalue_left_vertices_FPPs.astype(np.float32)

In [47]:
temp_l_path = 'D:/place_ROIs_project/Initial_ROIs_data/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)
for i in range(1,6):
    temp_l.darrays[0].data = corr_left_32[:,i-1]
    temp_l.to_filename(f'D:/place_ROIs_project/Phase1_ExtractingFinalROIs/2_Correlation_vertices_with_FPPs/CorrelationFPPsWithVertices{i}.lh.func.gii')

In [25]:
np.nan_to_num(correlation_left_vertices_FPPs, copy=False, nan=0)
np.nan_to_num(pvalue_left_vertices_FPPs, copy=False, nan=0)

array([[3.00534071e-213, 2.81098395e-245, 3.67182745e-193,
        3.52345891e-099, 1.04126060e-006],
       [2.93171881e-071, 1.30804231e-311, 1.35168820e-289,
        3.98668146e-198, 3.00392521e-047],
       [1.76571062e-087, 3.10963046e-052, 2.57022263e-033,
        6.22037987e-036, 1.70389417e-008],
       ...,
       [1.87987857e-067, 2.13922175e-039, 9.66328952e-001,
        4.99072668e-056, 1.39527882e-177],
       [2.26174587e-076, 1.81237928e-056, 1.72982849e-003,
        6.06945606e-032, 1.17762806e-119],
       [1.49592883e-039, 4.82013888e-035, 5.74828667e-001,
        6.84606903e-056, 1.14026189e-157]])

In [26]:
sorted_correlation_left_vertices_FPPs = np.sort(correlation_left_vertices_FPPs, axis=1)[:, ::-1]
sorted_pvalue_left_vertices_FPPs = np.sort(pvalue_left_vertices_FPPs, axis=1)[:, ::-1]
sorted_prefered_FPP_each_vertices_left = np.argsort(correlation_left_vertices_FPPs, axis=1)[:, ::-1]

In [27]:
preferred_FPP_each_vertices_left = sorted_prefered_FPP_each_vertices_left[:,0] + 1

In [28]:
positive_preferred_corr_each_vertices_left = sorted_correlation_left_vertices_FPPs[:,0][sorted_correlation_left_vertices_FPPs[:,0]>0]

In [29]:
mean_sorted_correlation_left_vertices_FPPs= np.mean(positive_preferred_corr_each_vertices_left) 

In [67]:
threshold = 0.3
above_threshold_vertices_left = sorted_correlation_left_vertices_FPPs[:,0] >= threshold

In [68]:
preferred_FPP_above_threshold_vertices_left = np.zeros_like(preferred_FPP_each_vertices_left)

preferred_FPP_above_threshold_vertices_left[above_threshold_vertices_left] = preferred_FPP_each_vertices_left[above_threshold_vertices_left]

In [69]:
temp_l_path = 'D:/place_ROIs_project/Initial_ROIs_data/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)
temp_l.darrays[0].data = preferred_FPP_above_threshold_vertices_left
temp_l.to_filename('D:/place_ROIs_project/Phase1_ExtractingFinalROIs/2_Correlation_vertices_with_FPPs/preferred_FPP_above_threshold_vertices.L.func.gii')